# Article Labeling & Lexical Fields Finding

In [14]:
import pandas as pd
import numpy as np
import sys
import re

## Download data

In [15]:
import json

data = []
with open('./data/20200420_20200714_business_articles.json') as f:
    for line in f:
        data.append(json.loads(line))


In [16]:
print ("data type",type (data))
print ("json",type (data[0]))
print ("keys",data[0].keys())
print ("length", len(data))

data type <class 'list'>
json <class 'dict'>
keys dict_keys(['published', 'link', 'message', 'Feed', 'title', '@version', 'author', '@timestamp', 'full-text', 'type'])
length 416307


## Extracting full_text of each article:

In [17]:
articles = list()
min_length = 1000000
min_article_size = 2000
for article in data:
    plain_text = article.get('full-text')
    if (plain_text and "Article `download()` failed" != plain_text[:27] and "Please enable cookies" != plain_text[:21] and len(plain_text)>min_article_size):
        articles.append(plain_text)
        # Check for small articles
        #if len(plain_text)<min_length:
        #    min_length = len(plain_text)
        #    print("*****************")
        #    print (plain_text)
        #    print("*****************")

In [18]:
# 358192 removing "Article `download()` failed" 
# 340987 removing "Article `download()` failed" and "Please enable cookies"
# 215039 removing "Article `download()` failed" and "Please enable cookies" and size<min_article_size = 2000
print ("len(articles)",len(articles) )
print ("min_length",min_length)

#for i in range (3001, 3002, 1):
#    print ("article", i)
#    print ("article", i, articles[i])


len(articles) 215039
min_length 1000000


In [190]:
# Cleaning full_text of articles
# Remove every non-letter character
clean_articles = list()
n_articles = 2000
for article in articles[:n_articles]:
    clean_text = article.lower()
    clean_text = re.sub(r'\s+', ' ', clean_text)
    clean_text = re.sub("[^a-z]", ' ', clean_text)
    clean_text = re.sub(r'\s+', ' ', clean_text)
    clean_articles.append(clean_text)

In [20]:
clean_articles[0]

'eliminated masterchef contestant harry foster has hit back at unfair criticism against judge melissa leong the show s first female judge has faced a barrage of trolling with haters taking aim at everything from her behaviour on set to her fashion sense despite being eliminated on tuesday night s episode harry had nothing but good things to say about the melbourne based food writer this could not be further from the truth eliminated masterchef australia contestant harry foster pictured has hit back at unfair criticism against judge melissa leong she s a queen i love her harry told huffpost australia she is energetic passionate and really just vibrant when asked about accusations melissa was rude and biased on the show he said this could not be further from the truth all three judges have received an overwhelmingly positive response from fans but melissa has copped a backlash from a vocal minority she s a queen the show s first female judge has faced a barrage of trolling with haters ta

## Python program to generate word vectors using Word2Vec 

In [29]:
# importing all necessary modules 
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize 
#nltk.download('punkt')
import warnings 
  
warnings.filterwarnings(action = 'ignore') 
  
import gensim 
from gensim.models import Word2Vec 

import sklearn
from sklearn.feature_extraction.text import CountVectorizer

import collections

In [30]:
# Apply 2 Word2Vec models to articles   

data = [] 
  
# iterate through each article in the file 
for i in clean_articles: 
    temp = [] 
    # tokenize the article into words 
    for j in word_tokenize(i): 
        temp.append(j.lower()) 
  
    data.append(temp) 

# Create CBOW model 
model1 = gensim.models.Word2Vec(data, min_count = 1,  
                              size = 100, window = 5) 
  
# Print results 
print("Cosine similarity between 'australia' " + 
               "and 'melbourne' - CBOW : ", 
    model1.similarity('melbourne', 'australia')) 

print(model1.wv.most_similar('melbourne'))
    

# Create Skip Gram model 
model2 = gensim.models.Word2Vec(data, min_count = 1, size = 100, 
                                             window = 5, sg = 4) 
  
# Print results 
print("Cosine similarity between 'australia' " +
          "and 'melbourne' - Skip Gram : ", 
    model2.similarity('melbourne', 'australia')) 
print(model2.wv.most_similar('melbourne'))


Cosine similarity between 'australia' and 'melbourne' - CBOW :  0.9941112
[('committee', 0.9950198531150818), ('classrooms', 0.9950098395347595), ('university', 0.9949347376823425), ('seconds', 0.9949148893356323), ('effect', 0.9948941469192505), ('process', 0.994837760925293), ('low', 0.9947916865348816), ('usa', 0.9947860240936279), ('each', 0.9947835803031921), ('chaos', 0.9947378635406494)]
Cosine similarity between 'australia' and 'melbourne' - Skip Gram :  0.99408007
[('nose', 0.9993689060211182), ('harassed', 0.9993646144866943), ('patients', 0.9993616342544556), ('compounds', 0.9993556141853333), ('professional', 0.9993484020233154), ('operation', 0.9993415474891663), ('reluctance', 0.9993293285369873), ('harsh', 0.9993184208869934), ('almost', 0.9993061423301697), ('fresh', 0.999296247959137)]


In [33]:
# FOR GENSIN USING CBOW Manipulations

# enumerate data it is trained on
for i, word in enumerate(model1.wv.vocab):
    if i == 5:
        break
    print(word)

eliminated
masterchef
contestant
harry
foster


In [35]:
len(data)
# Show frequencies
#print("Original List : ",data)
data_flat = []
for line in data:
    for word in line:
        data_flat.append(word)


ctr = collections.Counter(data_flat)
#print("Frequency of the elements in the List : ",ctr)
ctr["the"] # count of word "the"

4045

## Using tf.itf

In [191]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [192]:
# Vectorizer
vectorizer = TfidfVectorizer(use_idf=True)
X = vectorizer.fit_transform(clean_articles)

#print(vectorizer.get_feature_names()[:10])
#print(X.shape)
#print(vectorizer.get_stop_words())
#print(vectorizer.get_params(deep=True))

n_articles, n_distinct_words = X.shape
print(n_articles, n_distinct_words)

collect_word_importance = []
#place tf-idf values in a pandas data frame 
for tf_idf_vector_id in range(n_articles):
    
    tf_idf_vector=X[tf_idf_vector_id]
    #print (tf_idf_vector.todense().sum())
    #print (tf_idf_vector.T.todense())
    df = pd.DataFrame(tf_idf_vector.T.todense(), index=vectorizer.get_feature_names(), columns=["tfidf"]) 
    df_word_importance = df.sort_values(by=["tfidf"],ascending=False)
    word_importance_list = np.array(df_word_importance.index)
    collect_word_importance.append(word_importance_list)


2000 29466


In [193]:
# Each line corresponds to the highest scored words in the article of same index.
collect_word_importance = np.array(collect_word_importance)
collect_word_importance

array([['melissa', 'masterchef', 'leong', ..., 'findlay', 'findings',
        'zuocheng'],
       ['the', 'burglary', 'bail', ..., 'firmware', 'firms', 'zuocheng'],
       ['the', 'to', 'children', ..., 'flaring', 'flareups', 'zuocheng'],
       ...,
       ['the', 'to', 'in', ..., 'firestorm', 'fires', 'zuocheng'],
       ['the', 'to', 'in', ..., 'firestorm', 'fires', 'zuocheng'],
       ['the', 'to', 'in', ..., 'firestorm', 'fires', 'zuocheng']],
      dtype=object)

In [70]:
# TfidfTransformer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
import numpy as np

In [71]:
# TfidfTransformer
#TfidfTransformer(*, norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)[source]


corpus = ['this is the first document',
           'this document is the second document',
          'and this is the third one',
           'is this the first document']
vocabulary = ['this', 'document', 'first', 'is', 'second', 'the',
               'and', 'one']
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
                  ('tfid', TfidfTransformer())]).fit(corpus)
pipe['count'].transform(corpus).toarray()
pipe['tfid'].idf_
pipe.transform(corpus).shape


(4, 8)

In [91]:
pipe = Pipeline([('count', CountVectorizer()),
                  ('tfid', TfidfTransformer())]).fit(clean_articles)
pipe['count'].transform(clean_articles).toarray().shape
print (pipe['tfid'].idf_)
Tfidf_res = pipe.transform(clean_articles)
Tfidf_res.shape

[7.2156076  7.2156076  7.2156076  ... 6.81014249 7.2156076  6.52246042]


(1000, 21395)

In [96]:
Tfidf_res

<1000x21395 sparse matrix of type '<class 'numpy.float64'>'
	with 334368 stored elements in Compressed Sparse Row format>

In [98]:
#### Tutorial

#Dataset and Imports
import pandas as pd 
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer 
 
# this is a very toy example, do not try this at home unless you want to understand the usage differences 
docs=["the house had a tiny little mouse", 
"the cat saw the mouse", 
"the mouse ran away from the house", 
"the cat finally ate the mouse", 
"the end of the mouse story"
]

In [99]:
# Initialize CountVectorizer
#instantiate CountVectorizer() 
cv=CountVectorizer() 
# this steps generates word counts for the words in your docs 
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape
# 5 texts, 9 distinct words -> gives the count for each word in each text

(5, 16)

In [101]:
#Compute the IDF values
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [102]:
# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

,idf_weights
mouse,1.000000
the,1.000000
cat,1.693147
house,1.693147
ate,2.098612
away,2.098612
end,2.098612
finally,2.098612
from,2.098612
had,2.098612


In [104]:
# Compute the TFIDF score for your documents
# count matrix 
count_vector=cv.transform(docs) #<==> word_count_vector

# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)

In [105]:
tf_idf_vector

<5x16 sparse matrix of type '<class 'numpy.float64'>'
	with 26 stored elements in Compressed Sparse Row format>

In [108]:
feature_names = cv.get_feature_names() 
 
#get tfidf vector for FFFFFFFFFirst document 
first_document_vector=tf_idf_vector[0] 
 
#print the scores (Tf-idf scores of first document)
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
had,0.493562
little,0.493562
tiny,0.493562
house,0.398203
mouse,0.235185
the,0.235185
ate,0.000000
away,0.000000
cat,0.000000
end,0.000000


In [160]:
# Tfidfvectorizer Usage - Compute all at Once
from sklearn.feature_extraction.text import TfidfVectorizer 
 
# settings that you use for count vectorizer will go here 
tfidf_vectorizer=TfidfVectorizer(use_idf=True) 
#fitted_vectorizer=tfidf_vectorizer.fit(docs)               # This method would work too
#tfidf_vectorizer_vectors=fitted_vectorizer.transform(docs)  
 
# just send in all your docs here 
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(docs)

In [164]:
# get the first vector out (for the first document) 
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0] 
 
# place tf-idf values in a pandas data frame 
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)


,tfidf
had,0.493562
little,0.493562
tiny,0.493562
house,0.398203
mouse,0.235185
the,0.235185
ate,0.000000
away,0.000000
cat,0.000000
end,0.000000
